In [1]:
pip install yt-dlp assemblyai transformers torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Step 1: Import required libraries
import yt_dlp
import assemblyai as aai
from transformers import pipeline
import time


In [3]:
# Set API keys
ASSEMBLYAI_API_KEY = '452837939d2b4902808668041543d53d'  
aai.settings.api_key = ASSEMBLYAI_API_KEY


# Set your API keys
ASSEMBLYAI_API_KEY = 'e4de53d23ca8412090f6c64ccc1e5f91'  # Your AssemblyAI API key
aai.settings.api_key = ASSEMBLYAI_API_KEY


In [4]:
# Step 2: Download YouTube Audio using yt-dlp
import os 
def download_audio_from_youtube(video_url, output_file='audio', retries=3):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_file  # Let yt-dlp handle the extension
    }
    
    for attempt in range(retries):
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            return output_file + '.mp3'  # Return the correct file path
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise
            time.sleep(5)  # Wait before retrying

# Example usage
video_url = "https://youtu.be/DneLxrPmmsw?si=-q0wasqvtTZFTU5t"  # Replace with your video URL
audio_file = download_audio_from_youtube(video_url)
print("Audio downloaded:", audio_file)

# Step 3: Transcribe Audio using AssemblyAI
def transcribe_audio(file_path):
    # Check if the file exists before transcription
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
    
    transcriber = aai.Transcriber()
    transcript = transcriber.transcribe(file_path)

    # Wait for the transcription to complete
    while transcript.status not in {aai.TranscriptStatus.completed, aai.TranscriptStatus.error}:
        time.sleep(5)  # Wait a few seconds before checking the status again
        transcript = transcriber.get_transcript(transcript.id)  # Update the transcript status

    if transcript.status == aai.TranscriptStatus.error:
        raise Exception(f"Transcription failed: {transcript.error}")
    
    return transcript.text

# Transcribe the downloaded audio file
transcription = transcribe_audio(audio_file)
print("Transcription:", transcription)

if not transcription:
    raise ValueError("Transcription returned empty or invalid text.")

[youtube] Extracting URL: https://youtu.be/DneLxrPmmsw?si=-q0wasqvtTZFTU5t
[youtube] DneLxrPmmsw: Downloading webpage
[youtube] DneLxrPmmsw: Downloading ios player API JSON
[youtube] DneLxrPmmsw: Downloading web creator player API JSON
[youtube] DneLxrPmmsw: Downloading player 34059770
[youtube] DneLxrPmmsw: Downloading m3u8 information
[info] DneLxrPmmsw: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    4.68MiB in 00:00:01 at 3.42MiB/s   
[ExtractAudio] Destination: audio.mp3
Deleting original file audio (pass -k to keep)
Audio downloaded: audio.mp3
Transcription: In this presentation we will learn how to create a node of a single link list. So let's get started. First we will recall what is self referential structure. Self referential structure is a structure which contains a pointer to a structure of the same type. And here is an example of a self referential structure. I am considering here this struct Abc as a structure which contains a pointer to t

In [5]:
# Step 4: Summarize transcription using Hugging Face Transformers
def summarize_text(transcript):
    max_chunk_length = 1024  
    chunks = [transcript[i:i + max_chunk_length] for i in range(0, len(transcript), max_chunk_length)]
    
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    summaries = []
    
    for chunk in chunks:
        max_length = min(150, len(chunk) // 2)  
        summary = summarizer(chunk, max_length=max_length, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    
    return " ".join(summaries)  

# Get the summary of the transcription
summary = summarize_text(transcription)
print("Summary:", summary)


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mobar\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

KeyboardInterrupt: 

: 

In [13]:
# Save the summary to a text file
with open('summary.txt', 'w') as f:
    f.write(summary)
print("Summary saved to summary.txt")

Summary saved to summary.txt
